In [208]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_text
import tensorflow_hub as hub

from collections import defaultdict
from tqdm.notebook import tqdm

import os

In [247]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [262]:
sample_submission = pd.read_csv('/content/drive/MyDrive/sample.technosphere.ir1.textrelevance.submission.txt')
sample_submission

,QueryId,DocumentId
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
...,...,...
38767,399,38110
38768,399,38111
38769,399,38112
38770,399,38113


In [93]:
f = open('/content/drive/MyDrive/queries.numerate.txt', 'r')
queries = list(map(lambda x: x.split('\t')[-1].split(), f.read().split('\n')))

In [ ]:
file_name = '/content/drive/MyDrive/headers.zip'

with zipfile.ZipFile(file_name, 'r') as zip:
    zip.extractall()

In [88]:
files_names = os.listdir('headers')

In [89]:
files_words = []

for file_name in files_names:
  f = open('headers/{}'.format(file_name), 'r')
  files_words.append(f.read().split())

In [101]:
all_words = []

for query in queries:
  all_words.extend(query)

for doc_keys in files_words:
  all_words.extend(doc_keys)

In [138]:
idf_words = defaultdict(float)

for doc_keys in files_words:
  for key in set(doc_keys):
    idf_words[key] += 1

idf_words = {word: 1/idf_value for word, idf_value in idf_words.items()}
idf_words.update({word: 1.0 for word in all_words})

In [139]:
avg_doc_len = np.mean([len(line) for line in files_words])

9.041585769008764

In [175]:
def BM25_score(doc_id, query_id, k=2, b=0.75):

  bm25_score = 0.0
  doc = files_words[doc_id]
  query = queries[query_id]

  query_tf = defaultdict(float)
  for word in query:
      query_tf[word] = doc.count(word)
  
  for word in query:
    bm25_score += idf_words[word] * ((query_tf[word] * (k+1)) / (query_tf[word] + k*(1 - b + b * (len(doc)/avg_doc_len))))
  
  return bm25_score

In [184]:
doc_id_rand = np.random.randint(0,len(files_words))
query_id_rand = np.random.randint(0, len(queries))

bm25_score_dict = dict()
for doc_id in range(len(files_words)):
  bm25_score_dict.update({doc_id: BM25_score(doc_id, 0)})
  # print('doc_id', doc_id, ' - ', bm25_score_list[-1])

In [261]:
submission.query('QueryId == 1').iloc[:,1].values

array([], dtype=int64)

In [265]:
sub_list = []
for query_id in tqdm(range(0, len(queries))):

  bm25_score_dict = dict()
  for doc_id in range(len(files_words)):
    bm25_score_dict.update({doc_id: BM25_score(doc_id, query_id)})

  real_query_id = query_id + 1
  doc_ids = set(sample_submission.query('QueryId == @real_query_id').iloc[:,1].values)
  bm25_score_dict = dict(filter(lambda x: x[0] in doc_ids, bm25_score_dict.items()))  
  docs = list(map(lambda x: x[0], sorted(bm25_score_dict.items(), key=lambda x: x[1], reverse=True)))[:10]
  sub_list.extend([[real_query_id, docs[i]] for i in range(len(docs))])

In [ ]:
module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')

In [ ]:
question_embeddings = module.signatures['question_encoder'](
            tf.constant(questions))['outputs'].numpy()
response_embeddings = module.signatures['response_encoder'](
        input=tf.constant(responses),
        context=tf.constant(response_contexts))['outputs'].numpy()
cosine_sim(question_embeddings, response_embeddings)

In [ ]:
def arccosine_sim(emb1, emb2):
    sim = 1 - np.arccos(
      sklearn.metrics.pairwise.cosine_similarity(emb1,
                                                 emb2))/np.pi
    return sim

In [ ]:
poss = defaultdict(list)

for query_id in range(1, 400):
    rmpdata = samples.loc[samples['QueryId'] == query_id]
    tmp = 0
    for rw in rmpdata.iterrows():
        tmp += 1
        documentId = rw[1]['DocumentId']
        poss[query_id].append(documentId)
        if tmp == 25: break

In [ ]:
quaries_ids = []
docs = []

for query_id in tqdm(range(1, len(queries)+1)):

    lizt = poss[query_id]
    query = q_array[i-1]
    score_dict = {}

    for rel_doc in lizt:

        title = files_words[rel_doc]
        question_embeddings = module.signatures['question_encoder'](tf.constant(query))['outputs'].numpy()
        response_embeddings = module.signatures['response_encoder'](input=tf.constant(title), context=tf.constant(title))['outputs'].numpy()
        cos_sim = arccosine_sim(question_embeddings, response_embeddings)
        score_dict[rel_doc] = cos_sim

    sorted_values = sorted(score_dict, key=score_dict.get, reverse=True)
    docs.extend(sorted_values)

    for _ in range(len(sorted_values)):
        quaries_ids.append(query_id)

In [ ]:
submission = pd.DataFrame({'QueryId' : quaries_ids, 'DocumentId' : docs})
submission.to_csv('new_try_submission.csv', index=False)

In [ ]:
# submission = pd.DataFrame({'QueryId': np.array(sub_list)[:,0], 'DocumentId': np.array(sub_list)[:,1]})
# submission
# submission.to_csv('new_try_submission.csv', index=False)